In [25]:
from generate_signal import generate_X_matrix
from beamforming import beamforming_method
from generate_signal import generate_A_matrix
from generate_signal import generate_noise
from generate_signal import generate_R_hat
from music import music_method
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np


nbSources = 2 # Nombre de sources
nbSensors = 10 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
var1 = 1 # Variance du signal 1
var2 = 1 # Variance du signal 2
L = 10
T = 100
correlation_List = [0.4] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.


In [36]:
num_samples = 1000000  # Nombre d'échantillons à générer

# Initialiser les tableaux pour stocker les matrices de covariance et les angles estimés
all_R_hat = np.zeros((num_samples, L, L), dtype=complex)
all_estimated_angles = []
for i in range(num_samples):
    # Générer des angles aléatoires pour chaque échantillon
    theta1 = np.random.uniform(-90, 90)
    theta2 = np.random.uniform(-90, 90)
    var1 = np.random.uniform(0,1000000000000)
    var2 = np.random.uniform(0,1000000000000)
    thetaList = [theta1, theta2]
    varList = [var1, var2]

    # Générer la matrice X
    X = generate_X_matrix(nbSources, L, T, thetaList, varList, correlation_List, signal_noise_ratio)

    # Calcul de la matrice de covariance pour l'échantillon actuel
    R_hat = generate_R_hat(X)
    all_R_hat[i] = R_hat

    # Calcul du spectre MUSIC pour l'échantillon actuel
    estimated_angles = music_method(X, L, T, print_angles = False, draw_plot = False)
    all_estimated_angles.append(estimated_angles)

# Division des données en ensembles d'entraînement et de test
all_estimated_angles = np.array(all_estimated_angles)  # Conversion en numpy array si nécessaire
R_hat_train, R_hat_test, angles_train, angles_test = train_test_split(
    all_R_hat, all_estimated_angles, test_size=0.2, random_state=42
)

[[1.61814163e+11 1.09798285e+11]
 [1.09798285e+11 4.65644628e+11]]
[[2.33899893e+10 5.66656430e+10]
 [5.66656430e+10 8.58004643e+11]]
[[5.65017564e+11 2.79869969e+11]
 [2.79869969e+11 8.66424391e+11]]
[[7.09397901e+11 2.97056181e+11]
 [2.97056181e+11 7.77440757e+11]]
[[5.93276790e+11 1.50661277e+11]
 [1.50661277e+11 2.39125531e+11]]
[[7.60349904e+11 3.08265159e+11]
 [3.08265159e+11 7.81115770e+11]]
[[6.87402674e+11 1.02428944e+11]
 [1.02428944e+11 9.53924905e+10]]
[[2.46393570e+11 1.29427615e+11]
 [1.29427615e+11 4.24917429e+11]]
[[5.82602187e+11 3.02927141e+11]
 [3.02927141e+11 9.84428726e+11]]
[[4.43016875e+10 7.22176657e+10]
 [7.22176657e+10 7.35777734e+11]]
[[6.63990522e+11 3.07526133e+11]
 [3.07526133e+11 8.90188937e+11]]
[[2.44149898e+11 1.50016257e+10]
 [1.50016257e+10 5.76102978e+09]]
[[3.72743573e+11 2.03726625e+11]
 [2.03726625e+11 6.95929802e+11]]
[[4.19583369e+10 7.14981980e+10]
 [7.14981980e+10 7.61468503e+11]]
[[6.91603174e+11 2.20311276e+11]
 [2.20311276e+11 4.38628139e+

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DOAEstimator(nn.Module):
    def __init__(self, input_size, output_size):
        super(DOAEstimator, self).__init__()
        # Définir les couches
        self.fc1 = nn.Linear(input_size, 1024) # Couche d'entrée à la première couche cachée
        self.fc2 = nn.Linear(1024, 512)        # Première couche cachée à la seconde
        self.fc3 = nn.Linear(512, output_size) # Seconde couche cachée à la couche de sortie

    def forward(self, x):
        # Passer les données à travers les couches
        x = F.relu(self.fc1(x))  # Activation ReLU pour la première couche cachée
        x = F.relu(self.fc2(x))  # Activation ReLU pour la seconde couche cachée
        x = self.fc3(x)          # Pas d'activation dans la couche de sortie pour la régression
        return x

# Création d'une instance du modèle avec les dimensions appropriées
# Par exemple, si la matrice de covariance est de taille 10x10 (aplatie à 100) et que vous prédisez un seul angle DoA
model = DOAEstimator(input_size=100, output_size=1).to(device)

# Affichage du modèle
print(model)



DOAEstimator(
  (fc1): Linear(in_features=100, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1, bias=True)
)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class DOADataset(Dataset):
    def __init__(self, covariance_matrices, doa_angles):
        self.covariance_matrices = covariance_matrices
        self.doa_angles = doa_angles

    def __len__(self):
        return len(self.covariance_matrices)

    def __getitem__(self, idx):
        cov_matrix = self.covariance_matrices[idx]
        angle = self.doa_angles[idx]

        # Conversion en tenseurs PyTorch
        cov_matrix = torch.from_numpy(np.real(cov_matrix)).float()
        angle = torch.tensor(angle).float()

        return cov_matrix, angle

# Création des instances de DOADataset
train_dataset = DOADataset(R_hat_train, angles_train)
test_dataset = DOADataset(R_hat_test, angles_test)

# Création des DataLoaders pour l'entraînement et le test
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [ ]:
import torch.optim as optim

# Configuration de l'optimiseur
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# Fonctions de boucle d'entraînement et d'évaluation
def train_loop(dataloader, model, loss_fn, optimizer):
    total_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        # Aplatir les données
        X_flattened = X.view(X.size(0), -1)

        # Compute prediction and loss
        pred = model(X_flattened)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{len(dataloader.dataset):>5d}]")

    return total_loss / len(dataloader)

def eval_loop(dataloader, model, loss_fn):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            X_flattened = X.view(X.size(0), -1)
            pred = model(X_flattened)
            total_loss += loss_fn(pred, y).item()

    avg_loss = total_loss / len(dataloader)
    print(f"Test Avg loss: {avg_loss:>8f} \n")
    return avg_loss

# Nombre d'époques
epochs = 10

# Boucle d'entraînement
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
    print(f"Training Loss: {train_loss}")
    test_loss = eval_loop(test_dataloader, model, loss_fn)
    print(f"Testing Loss: {test_loss}")
print("Done!")


Epoch 1
-------------------------------
loss: 75635685927475285590016.000000  [    0/   80]
Training Loss: 6.49746868271703e+22
Test Avg loss: 49119567681828746690560.000000 

Testing Loss: 4.911956768182875e+22
Epoch 2
-------------------------------
loss: 54086947998825856368640.000000  [    0/   80]
Training Loss: 4.853860483109783e+22
Test Avg loss: 42968699929553826480128.000000 

Testing Loss: 4.296869992955383e+22
Epoch 3
-------------------------------
loss: 52174597999854404763648.000000  [    0/   80]
Training Loss: 3.856406727929477e+22
Test Avg loss: 38438254344804574953472.000000 

Testing Loss: 3.8438254344804575e+22
Epoch 4
-------------------------------
loss: 36694710518866037964800.000000  [    0/   80]
Training Loss: 3.804204941698672e+22
Test Avg loss: 35263824593458070290432.000000 

Testing Loss: 3.526382459345807e+22
Epoch 5
-------------------------------
loss: 50680330665090638413824.000000  [    0/   80]
Training Loss: 3.381170494640699e+22
Test Avg loss: 3290